# Installation Infortmation

In [18]:
# type to install object-detection
# cd models/research/
# protoc object_detection/protos/*.proto --python_out=.
# cp object_detection/packages/tf2/setup.py .
# python -m pip install .
# Run "Download Tensorflow files and scripts" below

# !pip install opencv-python
# !pip install wget
# !pip install tensorflow==2.10.0
# !pip install tensorflow-gpu==2.10.0
# !pip install matplotlib
# !pip install tensorflow-addons
# !pip uninstall pycocotools -y
# !pip install pycocotools
# !pip install PyYAML
# !pip install gin
# !pip install gin-config

In [19]:
import os
import cv2 
import wget
import object_detection

import tensorflow as tf
import numpy as np

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from tensorflow.python.client import device_lib

from matplotlib import pyplot as plt
%matplotlib inline

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 

In [20]:
# print(device_lib.list_local_devices())

# Setup Paths

In [21]:
#    Features and Comparisons of Tensorflow Object Detection Algorithms
#    Referenced from:  https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
#    -----------------------------------------------------------------------------------------------------------------------
#
#    Model name	                                      |   Speed (ms)	  |   COCO mAP	   |  Outputs
#    -------------------------------------------------|-------------------|----------------|-----------------
#    CenterNet HourGlass104 512x512	                  |      70	          |   41.9	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    CenterNet HourGlass104 Keypoints 512x512	      |      76	          |   40.0/61.4	   |  Boxes/Keypoints
#    -------------------------------------------------|-------------------|----------------|-----------------
#    CenterNet HourGlass104 1024x1024	              |      197	      |   44.5	       |  Boxes             -
#    -------------------------------------------------|-------------------|----------------|-----------------
#    CenterNet HourGlass104 Keypoints 1024x1024	      |      211	      |   42.8/64.5	   |  Boxes/Keypoints
#    -------------------------------------------------|-------------------|----------------|-----------------
#    CenterNet Resnet50 V1 FPN 512x512	              |      27	          |   31.2	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    CenterNet Resnet50 V1 FPN Keypoints 512x512	  |      30	          |   29.3/50.7	   |  Boxes/Keypoints
#    -------------------------------------------------|-------------------|----------------|-----------------
#    CenterNet Resnet101 V1 FPN 512x512	              |      34	          |   34.2	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    CenterNet Resnet50 V2 512x512	                  |      27	          |   29.5	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    CenterNet Resnet50 V2 Keypoints 512x512	      |      30	          |   27.6/48.2	   |  Boxes/Keypoints
#    -------------------------------------------------|-------------------|----------------|-----------------
#    CenterNet MobileNetV2 FPN 512x512	              |      6	          |   23.4	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    CenterNet MobileNetV2 FPN Keypoints 512x512	  |      6	          |   41.7	       |  Keypoints
#    -------------------------------------------------|------------------ |----------------|-----------------
#    EfficientDet D0 512x512	                      |      39	          |   33.6	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    EfficientDet D1 640x640	                      |      54	          |   38.4	       |  Boxes             -
#    -------------------------------------------------|-------------------|----------------|-----------------
#    EfficientDet D2 768x768	                      |      67	          |   41.8	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    EfficientDet D3 896x896	                      |      95	          |   45.4	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    EfficientDet D4 1024x1024	                      |      133	      |   48.5	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    EfficientDet D5 1280x1280	                      |      222	      |   49.7	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    EfficientDet D6 1280x1280	                      |      268	      |   50.5	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    EfficientDet D7 1536x1536	                      |      325	      |   51.2	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    SSD MobileNet v2 320x320	                      |      19	          |   20.2	       |  Boxes             +
#    -------------------------------------------------|-------------------|----------------|-----------------
#    SSD MobileNet V1 FPN 640x640	                  |      48	          |   29.1	       |  Boxes             ++
#    -------------------------------------------------|-------------------|----------------|-----------------
#    SSD MobileNet V2 FPNLite 320x320	              |      22	          |   22.2	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    SSD MobileNet V2 FPNLite 640x640	              |      39	          |   28.2	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	      |      46	          |   34.3	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)	  |      87	          |   38.3	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    SSD ResNet101 V1 FPN 640x640 (RetinaNet101)      |      57	          |   35.6	       |  Boxes             -
#    -------------------------------------------------|-------------------|----------------|-----------------
#    SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)	  |      104	      |   39.5	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    SSD ResNet152 V1 FPN 640x640 (RetinaNet152)	  |      80	          |   35.4	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)	  |      111	      |   39.6	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Faster R-CNN ResNet50 V1 640x640	              |      53	          |   29.3	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Faster R-CNN ResNet50 V1 1024x1024	              |      65	          |   31.0	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Faster R-CNN ResNet50 V1 800x1333	              |      65	          |   31.6	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Faster R-CNN ResNet101 V1 640x640	              |      55	          |   31.8	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Faster R-CNN ResNet101 V1 1024x1024	          |      72	          |   37.1	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Faster R-CNN ResNet101 V1 800x1333	              |      77	          |   36.6	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Faster R-CNN ResNet152 V1 640x640	              |      64	          |   32.4	       |  Boxes             -
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Faster R-CNN ResNet152 V1 1024x1024	          |      85	          |   37.6	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Faster R-CNN ResNet152 V1 800x1333	              |      101	      |   37.4	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Faster R-CNN Inception ResNet V2 640x640	      |      206	      |   37.7         |  Boxes             -
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Faster R-CNN Inception ResNet V2 1024x1024	      |      236	      |   38.7	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    Mask R-CNN Inception ResNet V2 1024x1024	      |      301	      |   39.0/34.6	   |  Boxes/Masks
#    -------------------------------------------------|-------------------|----------------|-----------------
#    ExtremeNet (deprecated)	                      |      --	          |   --	       |  Boxes
#    -------------------------------------------------|-------------------|----------------|-----------------
#    ExtremeNet	                                      |      --	          |   --	       |  Boxes

In [22]:
URL_PreTrainedModel_CenterNet_HourGlass104_512x512 =                    'http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_Centernet_HourGlass104_Keypoints_512x512 =          'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz'
URL_PreTrainedModel_CenterNet_HourGlass104_1024x1024 =                  'http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_1024x1024_coco17_tpu-32.tar.gz'
URL_PreTrainedModel_CenterNet_HourGlass104_Keypoints_1024x1024 =        'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_1024x1024_kpts_coco17_tpu-32.tar.gz'
URL_PreTrainedModel_CenterNet_Resnet50_V1_FPN_512x512 =                 'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v1_fpn_512x512_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_CenterNet_Resnet50_V1_FPN_Keypoints_512x512 =       'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v1_fpn_512x512_kpts_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_CenterNet_Resnet101_V1_FPN_512x512 =                'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet101_v1_fpn_512x512_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_CenterNet_Resnet50_V2_512x512 =                     'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v2_512x512_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_CenterNet_Resnet50_V2_Keypoints_512x512 =           'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v2_512x512_kpts_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_CenterNet_MobileNetV2_FPN_512x512 =                 'http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz'
URL_PreTrainedModel_CenterNet_MobileNetV2_FPN_Keypoints_512x512 =       'http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_kpts.tar.gz'
URL_PreTrainedModel_EfficientDet_D0_512x512 =                           'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz'
URL_PreTrainedModel_EfficientDet_D1_640x640 =                           'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz'
URL_PreTrainedModel_EfficientDet_D2_768x768 =                           'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d2_coco17_tpu-32.tar.gz'
URL_PreTrainedModel_EfficientDet_D3_896x896 =                           'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d3_coco17_tpu-32.tar.gz'
URL_PreTrainedModel_EfficientDet_D4_1024x1024 =                         'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz'
URL_PreTrainedModel_EfficientDet_D5_1280x1280 =                         'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d5_coco17_tpu-32.tar.gz'
URL_PreTrainedModel_EfficientDet_D6_1280x1280 =                         'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d6_coco17_tpu-32.tar.gz'
URL_PreTrainedModel_EfficientDet_D7_1536x1536 =                         'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d7_coco17_tpu-32.tar.gz'
URL_PreTrainedModel_SSD_MobileNet_v2_320x320 =                          'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_SSD_MobileNet_v1_FPN_640x640 =                      'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_SSD_MobileNet_v2_FPNLite_320x320 =                  'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_SSD_MobileNet_v2_FPNLite_640x640 =                  'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_SSD_ResNet50_v1_FPN_640x640_RetinaNet50 =           'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_SSD_ResNet50_v1_FPN_1024x1024_RetinaNet50 =         'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_SSD_ResNet101_v1_FPN_640x640_RetinaNet101 =         'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_SSD_ResNet101_v1_FPN_1024x1024_RetinaNet101 =       'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_SSD_ResNet152_v1_FPN_640x640_RetinaNet152 =         'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_640x640_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_SSD_ResNet152_v1_FPN_1024x1024_RetinaNet152 =       'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_Faster_RCNN_ResNet50_V1_640x640 =                   'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_Faster_RCNN_ResNet50_V1_1024x1024 =                 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_Faster_RCNN_ResNet50_V1_800x1333 =                  'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8.tar.gz'
URL_PreTrainedModel_Faster_RCNN_ResNet101_V1_640x640 =                  'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_640x640_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_Faster_RCNN_ResNet101_V1_1024x1024 =                'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_Faster_RCNN_ResNet101_V1_800x1333 =                 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8.tar.gz'
URL_PreTrainedModel_Faster_RCNN_ResNet152_V1_640x640 =                  'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_Faster_RCNN_ResNet152_V1_1024x1024 =                'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_Faster_RCNN_ResNet152_V1_800x1333 =                 'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8.tar.gz'
URL_PreTrainedModel_Faster_RCNN_Inception_ResNet_V2_640x640 =           'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_Faster_RCNN_Inception_ResNet_V2_1024x1024 =         'http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8.tar.gz'
URL_PreTrainedModel_Mask_RCNN_Inception_ResNet_V2_1024x1024 =           'http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz'
URL_PreTrainedModel_ExtremeNet_Deprecated =                             'http://download.tensorflow.org/models/object_detection/tf2/20200711/extremenet.tar.gz'
URL_PreTrainedModel_ExtremeNet =                                        'http://download.tensorflow.org/models/object_detection/tf2/20210507/extremenet.tar.gz'

In [23]:
Name_PreTrainedModel_CenterNet_HourGlass104_512x512 =                    'centernet_hg104_512x512_coco17_tpu-8'
Name_PreTrainedModel_Centernet_HourGlass104_Keypoints_512x512 =          'centernet_hg104_512x512_kpts_coco17_tpu-32'
Name_PreTrainedModel_CenterNet_HourGlass104_1024x1024 =                  'centernet_hg104_1024x1024_coco17_tpu-32'
Name_PreTrainedModel_CenterNet_HourGlass104_Keypoints_1024x1024 =        'centernet_hg104_1024x1024_kpts_coco17_tpu-32'
Name_PreTrainedModel_CenterNet_Resnet50_V1_FPN_512x512 =                 'centernet_resnet50_v1_fpn_512x512_coco17_tpu-8'
Name_PreTrainedModel_CenterNet_Resnet50_V1_FPN_Keypoints_512x512 =       'centernet_resnet50_v1_fpn_512x512_kpts_coco17_tpu-8'
Name_PreTrainedModel_CenterNet_Resnet101_V1_FPN_512x512 =                'centernet_resnet101_v1_fpn_512x512_coco17_tpu-8'
Name_PreTrainedModel_CenterNet_Resnet50_V2_512x512 =                     'centernet_resnet50_v2_512x512_coco17_tpu-8'
Name_PreTrainedModel_CenterNet_Resnet50_V2_Keypoints_512x512 =           'centernet_resnet50_v2_512x512_kpts_coco17_tpu-8'
Name_PreTrainedModel_CenterNet_MobileNetV2_FPN_512x512 =                 'centernet_mobilenetv2fpn_512x512_coco17_od'
Name_PreTrainedModel_CenterNet_MobileNetV2_FPN_Keypoints_512x512 =       'centernet_mobilenetv2fpn_512x512_coco17_kpts'
Name_PreTrainedModel_EfficientDet_D0_512x512 =                           'efficientdet_d0_coco17_tpu-32'
Name_PreTrainedModel_EfficientDet_D1_640x640 =                           'efficientdet_d1_coco17_tpu-32'
Name_PreTrainedModel_EfficientDet_D2_768x768 =                           'efficientdet_d2_coco17_tpu-32'
Name_PreTrainedModel_EfficientDet_D3_896x896 =                           'efficientdet_d3_coco17_tpu-32'
Name_PreTrainedModel_EfficientDet_D4_1024x1024 =                         'efficientdet_d4_coco17_tpu-32'
Name_PreTrainedModel_EfficientDet_D5_1280x1280 =                         'efficientdet_d5_coco17_tpu-32'
Name_PreTrainedModel_EfficientDet_D6_1280x1280 =                         'efficientdet_d6_coco17_tpu-32'
Name_PreTrainedModel_EfficientDet_D7_1536x1536 =                         'efficientdet_d7_coco17_tpu-32'
Name_PreTrainedModel_SSD_MobileNet_v2_320x320 =                          'ssd_mobilenet_v2_320x320_coco17_tpu-8'
Name_PreTrainedModel_SSD_MobileNet_v1_FPN_640x640 =                      'ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8'
Name_PreTrainedModel_SSD_MobileNet_v2_FPNLite_320x320 =                  'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
Name_PreTrainedModel_SSD_MobileNet_v2_FPNLite_640x640 =                  'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
Name_PreTrainedModel_SSD_ResNet50_v1_FPN_640x640_RetinaNet50 =           'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
Name_PreTrainedModel_SSD_ResNet50_v1_FPN_1024x1024_RetinaNet50 =         'ssd_resnet50_v1_fpn_1024x1024_coco17_tpu-8'
Name_PreTrainedModel_SSD_ResNet101_v1_FPN_640x640_RetinaNet101 =         'ssd_resnet101_v1_fpn_640x640_coco17_tpu-8'
Name_PreTrainedModel_SSD_ResNet101_v1_FPN_1024x1024_RetinaNet101 =       'ssd_resnet101_v1_fpn_1024x1024_coco17_tpu-8'
Name_PreTrainedModel_SSD_ResNet152_v1_FPN_640x640_RetinaNet152 =         'ssd_resnet152_v1_fpn_640x640_coco17_tpu-8'
Name_PreTrainedModel_SSD_ResNet152_v1_FPN_1024x1024_RetinaNet152 =       'ssd_resnet152_v1_fpn_1024x1024_coco17_tpu-8'
Name_PreTrainedModel_Faster_RCNN_ResNet50_V1_640x640 =                   'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
Name_PreTrainedModel_Faster_RCNN_ResNet50_V1_1024x1024 =                 'faster_rcnn_resnet50_v1_1024x1024_coco17_tpu-8'
Name_PreTrainedModel_Faster_RCNN_ResNet50_V1_800x1333 =                  'faster_rcnn_resnet50_v1_800x1333_coco17_gpu-8'
Name_PreTrainedModel_Faster_RCNN_ResNet101_V1_640x640 =                  'faster_rcnn_resnet101_v1_640x640_coco17_tpu-8'
Name_PreTrainedModel_Faster_RCNN_ResNet101_V1_1024x1024 =                'faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8'
Name_PreTrainedModel_Faster_RCNN_ResNet101_V1_800x1333 =                 'faster_rcnn_resnet101_v1_800x1333_coco17_gpu-8'
Name_PreTrainedModel_Faster_RCNN_ResNet152_V1_640x640 =                  'faster_rcnn_resnet152_v1_640x640_coco17_tpu-8'
Name_PreTrainedModel_Faster_RCNN_ResNet152_V1_1024x1024 =                'faster_rcnn_resnet152_v1_1024x1024_coco17_tpu-8'
Name_PreTrainedModel_Faster_RCNN_ResNet152_V1_800x1333 =                 'faster_rcnn_resnet152_v1_800x1333_coco17_gpu-8'
Name_PreTrainedModel_Faster_RCNN_Inception_ResNet_V2_640x640 =           'faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8'
Name_PreTrainedModel_Faster_RCNN_Inception_ResNet_V2_1024x1024 =         'faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8'
Name_PreTrainedModel_Mask_RCNN_Inception_ResNet_V2_1024x1024 =           'mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8'
Name_PreTrainedModel_ExtremeNet_Deprecated =                             'extremenet_deprecated'
Name_PreTrainedModel_ExtremeNet =                                        'extremenet'

In [24]:
PRETRAINED_MODEL_NAME = Name_PreTrainedModel_SSD_MobileNet_v1_FPN_640x640
PRETRAINED_MODEL_URL = URL_PreTrainedModel_SSD_MobileNet_v1_FPN_640x640
CUSTOM_MODEL_NAME = 'my_tfcod_model' 
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [25]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [26]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [27]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [28]:
# This array is used to create the folder structure where the objects to be predicted will be stored.
# e.g.  labels = ['thumbsup', 'thumbsdown', 'thankyou', 'livelong']
labels = ['thumbsup', 'thumbsdown', 'thankyou', 'livelong']

In [29]:
ORIGINAL_IMAGES_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'Original_Images')

In [30]:
if not os.path.exists(ORIGINAL_IMAGES_PATH):
    if os.name == 'posix':
        !mkdir -p {ORIGINAL_IMAGES_PATH}
    if os.name == 'nt':
         !mkdir {ORIGINAL_IMAGES_PATH}
for label in labels:
    path = os.path.join(ORIGINAL_IMAGES_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}

# Download Tensorflow files and scripts

In [33]:
# if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
#     !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [34]:
# # Install Tensorflow Object Detection 
# if os.name=='posix':  
#     !apt-get install protobuf-compiler
#     !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
# if os.name=='nt':
#     url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
#     wget.download(url)
#     !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
#     !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
#     os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
#     !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
#     !cd Tensorflow/models/research/slim && pip install -e . 

# Run Verification Script to Check Installation

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

# Create Label Map

In [ ]:
# e.g.  labels = [{'name':'thumbsup', 'id':1}, {'name':'thumbsdown', 'id':2}, {'name':'thankyou', 'id':3}, {'name':'livelong', 'id':4}]
labels = [{'name':'thumbsup', 'id':1}, {'name':'thumbsdown', 'id':2}, {'name':'thankyou', 'id':3}, {'name':'livelong', 'id':4}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Create TF records

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

# Download Pre-Trained Model

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# Copy Model Config to Training Folder

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# Update Config File

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# Train the Model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
print(command)

# Evaluate the Model

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [ ]:
print(command)

# Load Train Model From Checkpoint

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Detect from an Image

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'train', '{input_filename}')
savecv = "{output_filename}"

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
cv2.imwrite(savecv, image_np_with_detections)
plt.show()